In [1]:
import os.path
import datetime
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES=["https://www.googleapis.com/auth/calendar"]

In [7]:
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "google_desktop_credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

try:
    service = build("calendar", "v3", credentials=creds)
    # Create event
    event = {
        "summary" : "My event",
        "location" : "Patras",
        "description" : "My event is awesome",
        "colorId" : 6,
        "start" : {
            "dateTime":"2025-02-09T21:18:00+02:00",
            "timeZone":"Europe/Athens"
        },
        "end" : {
            "dateTime":"2025-02-09T21:50:00+02:00",
            "timeZone":"Europe/Athens"
        },
        "recurrence" : [
            "RRULE:FREQ=DAILY;COUNT=3"
        ]
    }
    event = ""
    event = service.events().insert(calendarId="primary", body=event).execute()
    # End of event creation
    # Call the Calendar API - Get 10 events
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])
        
except HttpError as error:
    print(f"An error occurred: {error}")

An error occurred: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Missing end time.". Details: "[{'domain': 'global', 'reason': 'required', 'message': 'Missing end time.'}]">
